# Dashboard Component: Fundamental Deep Dive

## Purpose
This query assesses the financial health of the company using balance sheet and income statement ratios.

## Indicators
*   **Solvency (Debt-to-Equity):** Measures long-term financial stability and bankruptcy risk.
*   **Liquidity (Current Ratio):** Measures the ability to pay short-term obligations.
*   **Moat (Gross Margin):** Indicates pricing power and competitive advantage.

In [0]:
CREATE OR REPLACE VIEW view_dashboard_fundamentals AS
SELECT 
    symbol,
    date as report_date,
    
    -- 1. Solvency (Can they pay long-term debt?)
    -- Debt-to-Equity Ratio
    round(total_debt / NULLIF(shares_outstanding * 150, 0), 2) as debt_to_equity, -- Using 150 as mock share price for equity calc
    CASE 
        WHEN (total_debt / NULLIF(shares_outstanding * 150, 0)) > 2.0 THEN '⚠️ Debt Overload (Risk of Bankruptcy)'
        WHEN (total_debt / NULLIF(shares_outstanding * 150, 0)) > 1.0 THEN '⚠️ Leveraged'
        ELSE '✅ Solid Foundation'
    END as solvency_intel,
    
    -- 2. Liquidity (Can they pay bills tomorrow?)
    round(liquidity_ratio, 2) as current_ratio,
    CASE 
        WHEN liquidity_ratio < 1.0 THEN '🚨 Cash Crunch (Liquidity Crisis)'
        WHEN liquidity_ratio < 1.5 THEN '⚠️ Tight Budget'
        ELSE '✅ Cash Rich'
    END as liquidity_intel,
    
    -- 3. Profitability (Quality of Earnings)
    round(gross_margin * 100, 1) as gross_margin_pct,
    CASE 
        WHEN gross_margin > 0.5 THEN '💎 Money Printer (High Moat)'
        WHEN gross_margin < 0.2 THEN '🍔 Commodity Business (Low Moat)'
        ELSE '🏭 Standard Business'
    END as moat_intel,
    
    -- 4. FCF Power
    calculated_fcf as free_cash_flow
FROM workspace.default.gold_valid_audit